/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
from ctransformers import AutoModelForCausalLM,AutoConfig,AutoTokenizer
from transformers import  TextIteratorStreamer
import torch
import gradio as gr
from threading import Thread


hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct-GGUF"
model_file = "Mistral-7b-instruct-cairo-instruct.Q4_k.gguf"
DEVICE,hw,layers = ("cuda",True,50) if torch.cuda.is_available() else ("cpu",False,0)


def load_ll():
    print("loading LLM")
    # Load model 
    config = AutoConfig.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF")

    config.max_seq_len = 4096
    config.max_answer_len= 1024

    llm = AutoModelForCausalLM.from_pretrained(hub_name, model_file=model_file, model_type="mistral", gpu_layers=layers,
            config=config,
                compress_pos_emb=2,
                top_k=4000,
                top_p=0.99,
                temperature=0.001,
                do_sample=True,
                hf=True

    )
    tokenizer = AutoTokenizer.from_pretrained(llm)
    return llm, tokenizer

def fmt_history(history) -> str:
    
    return "\n".join(["User: \"{usr_query}\", Assistant: \"{your_resp}\"".format(
        usr_query=usr_query.replace("\n",""), your_resp=your_resp.format("\n",""))
        for usr_query, your_resp in history])

    

def llm_func(message, chat_history=[]):
    text =f"""[INST]
        <<SYS>>
        Provide a concise answer to the questions<SYS>>

        Question: I'm working in Cairo 1 :{message} 
        [/INST]"""
  
    model_inputs = tokenizer([text], return_tensors="pt").to(DEVICE)

    # Start generation on a separate thread, so that we don't block the UI. The text is pulled from the streamer
    # in the main thread. Adds timeout to the streamer to handle exceptions in the generation thread.
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=float(temperature),
        top_k=top_k
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    # Pull the generated text from the streamer, and update the model output.
    model_output = ""
    for new_text in streamer:
        model_output += new_text
        yield model_output
    return model_output

    
title = 'Mistral Cairo 1.0 GGUF'
examples = [
    'Create an array and append domestic animal names',
    'How can you print a variable\'s value in Cairo? Give a full sample',
    ]
LLM,tokenizer = load_ll()

llm_func("Create an array and append domestic animal names")

loading LLM


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

NameError: name 'pouet' is not defined

In [29]:
prompt = "Create an array and append domestic animal names"



prompt = """
translate this Solidity contract into Cairo 1, and explain what you have done :
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title SimpleStorage
 * @dev Store & retrieve value in a variable
 */
contract SimpleStorage {
    uint256 private storedData;

    function store(uint256 num) public {
        storedData = num;
    }


    function retrieve() public view returns (uint256){
        return storedData;
    }
}

 """


text =f"""[INST]
<<SYS>>
 Provide a concise answer to the questions<SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

print(llm(text))



```cairo
#[derive(Copy, Drop)]
struct SimpleStorage {
    stored_data: u256
}

impl SimpleStorage {
    fn store(self, num: u256) {
        self.stored_data = num;
    }

    fn retrieve(self) -> u256 {
        self.stored_data
    }
}
```



In [31]:
from transformers import TextStreamer, GenerationConfig


prompt = "Create an array and append domestic animal names"
#prompt = "give an exemple of a simplestorage contract with unit tests"
#prompt = "create a felt and affect it a value of 1"
#prompt="In the example 'let y = { let x = 3; x + 1 };', what is the value of y? give a similar sample"
#prompt="How can you print a variable's value in Cairo? Give a full sample using an array"
#prompt = "hat type of operations can be considered expressions in Cairo? Give an exemple."
#prompt = "What is a characteristic feature of Felt252Dict<T> when interacting with it? Give a sample"
#prompt = "How do I create a span ? "
#prompt = "How do I know if an array is empty, give 2 examples"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt="write a contract that computes the fibonacci of caller's address and explain the weakness of the program"
#prompt="write a contract that returns the fibonacci of a value passed as parameter. implement the test function for the value 10"
#prompt="write an empty contract template and give the instructions to compile and deploy it"
#prompt="write a contract that stores the caller's address in its constructor and a function that returns the fibonnacci of that address"
#prompt="How can you implement the `Copy` trait on a custom type in Cairo?"
#prompt = "What's the significance of the `self` parameter in my method of the `CubeGeom` trait?"
#prompt = "write a contract with a function that returns the address of the caller"
#prompt="how can i cast a variable into another ?"

#prompt = "What do the terms 'parent' and 'child' mean in the context of modules?"
#prompt="What can be defined in trait and impl blocks related to a type?"
#prompt = "How do I know if an array is empty, give 2 examples"

#rompt="write a  basic simple contract that returns the fibonacci of the caller's address"
#prompt = "Create an array and append some domestic animal names as string"



#prompt = "How do I know if an array is empty, give 2 examples "

#prompt = "Provide a basic code snippet that defines a struct named 'User' with fields like 'active', 'username', and so on"


#prompt="write an empty contract template and give the instructions to compile and deploy it"

#prompt = "write a contract with a function that returns the address of the caller"
#prompt = "write a contract with a add_felt function that gets the sum of 2 felt parameters, add tests"
#prompt = "Create an array and append 3 domestic animal names as string and print the the second element"
#prompt = "write a contract that returns the fibonacci of the caller address"

prompt="write a  contract that provides a function that takes two arrays as parameter and store an array composed of the sum of the element of the arrays"
#prompt = "What's the difference between Nullable<T> and Option in terms of where the wrapped value is stored?"


#prompt = "How does the Cairo convention style name functions and variables?"

#prompt = "write a  contract that returns the fibonacci(caller address)"

#prompt = "What does the #[starknet::component] attribute signify?"

#prompt = " What is the naming convention for enum variants, give some examples"

#prompt ="""#[test]
#[available_gas(200000)]
#fn test_loop() {
#    let mut counter = 0;
#    //TODO make the test pass without changing any existing line
#    loop {
#        counter += 1;
#    };
#    assert(counter == 140, 'counter should be 140')
#}
#"""

#prompt = "write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter"
prompt="write an empty contract template and give the instructions to compile and deploy"

prompt = """
translate this Solidity contract into Cairo 1, and explain what you have done :
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title SimpleStorage
 * @dev Store & retrieve value in a variable
 */
contract SimpleStorage {
    uint256 private storedData;

    function store(uint256 num) public {
        storedData = num;
    }


    function retrieve() public view returns (uint256){
        return storedData;
    }
}

 """

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""



input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
                       # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1050,

            top_k=4000,

            top_p=0.99,
            do_sample=True,


            temperature=0.4,

            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            pad_token_id=model.config.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

NameError: name 'tokenizer' is not defined